In [1]:
import pandas as pd
import numpy as np
from statistics import mean
import sklearn
from sklearn.cluster import KMeans
from pandas import DataFrame
import csv

In [4]:
def ITD_mean_cal(l):
    x=mean(l)
    return x

def Refined_mean(l):
    ITD_collect=[]
    kmeans = KMeans(n_clusters=2, random_state=0).fit(l)
    X=kmeans.labels_
    #print(X)
    one_count = np.count_nonzero(X == 1)
    zero_count=np.count_nonzero(X == 0)
    #print("one_count=",one_count,",zero_count=",zero_count)
    if(one_count>=zero_count):
        index_list= np.where(X == 1)
        count=one_count
    else:
        index_list= np.where(X == 0)
        count=zero_count
    for i in range(count):
        #print(type(index_list[0][i].astype(np.int)))\
        ITD_collect.append(l[np.int16(index_list[0][i]).item()])
    #print(ITD_collect)
    ITD_collect=np.array(ITD_collect)
    mean=np.mean(ITD_collect)
    return mean


#  Raw data processing
header='MSI,RMSI,first,second,third,label\n'
usr_dir="Data_files/"
for user in range(21,22,1):
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    
    data=[]
    happy_bucket=[]
    sad_bucket=[]
    stressed_bucket=[]
    relax_bucket=[]
    
    print(usr_file)
    dataset=pd.read_csv(usr_file,header=None)
    dataframe=dataset.values
    
    
    # value list holds unique id 
    value_list=[]
    for i in range(dataframe.shape[0]):
        value_list.append(dataframe[i][0])
        
    ID_list=list(set(value_list))
    ID_list.sort()
    print("number of unique ID="+str(len(ID_list)))
    start=0
    next_start=0
    # traverse the whole list and add list of ITD values in emotion list
    for i in range(len(ID_list)):
        ID_number=ID_list[i]
        #print("ID_number="+str(ID_number))
        ITD_values=[]
        #### for time domain feature took character list #####
        char_list=[]
        for j in range(start,dataframe.shape[0],1):
            if(dataframe[j][0]==ID_number):
                ITD_values.append(dataframe[j][2])
                char_list.append(dataframe[j][3])
                next_start=next_start+1
            else:
                start=next_start
                #print(next_start)
                break
        if(dataframe[j-1][4]==2 and len(ITD_values)>3):
            happy_bucket.append(list(ITD_values))
            
            
            
        if(dataframe[j-1][4]==-2 and len(ITD_values)>3):
            sad_bucket.append(list(ITD_values))
            
            
        if(dataframe[j-1][4]==1 and len(ITD_values)>3):
            stressed_bucket.append(list(ITD_values))
            
            
        if(dataframe[j-1][4]==0 and len(ITD_values)>3):
            relax_bucket.append(list(ITD_values))
            
        #print("ID_number="+str(ID_number)+","+"length of list="+str(len(ITD_values)))
    print("Happy_bucket_length=",len(happy_bucket),"sad_bucket_length=",len(sad_bucket))  
    print("Stressed_bucket_length=",len(stressed_bucket),"Relaxed_bucket_length=",len(relax_bucket))   
    #print("sum of bucket length="+str(len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)))
    #count_ITD_list=len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)
    
    for itd_list in happy_bucket:
        
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        df = DataFrame (itd_list,columns=['ITD'])
        description = df.describe()
        list_value=np.reshape(description.loc['25%':'75%',:].values,3)
        list_value=list_value.tolist()
        temp=time_list+list_value
        temp.append("Happy")
        data.append(temp)
        
    for itd_list in sad_bucket:
        
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        df = DataFrame (itd_list,columns=['ITD'])
        description = df.describe()
        list_value=np.reshape(description.loc['25%':'75%',:].values,3)
        list_value=list_value.tolist()
        temp=time_list+list_value
        temp.append("Sad")
        data.append(temp) 
    
    for itd_list in stressed_bucket:
        
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        df = DataFrame (itd_list,columns=['ITD'])
        description = df.describe()
        list_value=np.reshape(description.loc['25%':'75%',:].values,3)
        list_value=list_value.tolist()
        temp=time_list+list_value
        temp.append("Stressed")
        data.append(temp)
        
    for itd_list in relax_bucket:
        
        l=np.array(itd_list)
        l=np.reshape(l,(len(l),1))
        time_list=[ITD_mean_cal(itd_list),Refined_mean(l)]
        df = DataFrame (itd_list,columns=['ITD'])
        description = df.describe()
        list_value=np.reshape(description.loc['25%':'75%',:].values,3)
        list_value=list_value.tolist()
        temp=time_list+list_value
        temp.append("Relaxed")
        data.append(temp)   
        
    writefile="SMOTE_folder/"+"U"+str(user)+"_LIHF_itd_file.csv"
    with open(writefile,"w") as resultfile:
            resultfile.write(header)
            writer=csv.writer(resultfile)
            writer.writerows(data)    

Data_files/U21_LIHF_itd_file.csv
number of unique ID=115
Happy_bucket_length= 36 sad_bucket_length= 5
Stressed_bucket_length= 9 Relaxed_bucket_length= 59
